In [132]:
import os
import numpy as np
from sklearn.preprocessing import StandardScaler

import torch
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

In [133]:
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [134]:
class FahrenheitTemperatures(Dataset):
    def __init__(self, start=-212, stop=10214, step=2):
        super(FahrenheitTemperatures, self).__init__()
        
        # Intialize local variables and scale the data
        scaler_f = StandardScaler()
        scaler_c = StandardScaler()
        f_temp = np.arange(start, stop, step).reshape(-1, 1)
        c_temp = np.array([self._f2c(f) for f in f_temp]).reshape(-1, 1)
        
        # transform data 
        f_temp = scaler_f.fit_transform(f_temp)
        c_temp = scaler_c.fit_transform(c_temp)
        
        # convert to tensors
        self.X = torch.from_numpy(f_temp).float()
        self.y = torch.from_numpy(c_temp).float()
        self._samples = self.X.shape[0]
        
    def __getitem__(self, index):
        # support indexing such that dataset[i] can be used to get i-th sample
        # implement this python function for indexing
        return self.X[index], self.y[index] 
        
    def __len__(self):
        # we can call len(dataset) to return the size, so this can be used
        # as an iterator
        return self._samples
    
    def _f2c(self, f) -> float:
        return (f - 32) * 5.0/9.0
    
    @property
    def samples(self):
        return self._samples

In [135]:
class LinearNN(torch.nn.Module):
    def __init__(self, input_size, output_size, hidden_size):
        super(LinearNN, self).__init__()
        
        # Input, output, and hidden size paramaters
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        
        # Build the NN architecture
        self.model = torch.nn.Sequential(
            torch.nn.Linear(input_size, hidden_size),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_size, 1),
            torch.nn.ReLU())
        
    def forward(self, x):
        out = self.model(x)
        return out    

In [136]:
# Let's now access our dataset
dataset = FahrenheitTemperatures()
first_dataset = dataset[0]
features, labels = first_dataset
samples = dataset.samples
print('Fahrenheit: {:.4f}'.format(features[0]))
print('Celcius   : {:.4f}'.format(labels[0]))
print('Samples   : {:.2f}'.format(samples))

Fahrenheit: -1.7317
Celcius   : -1.7317
Samples   : 5213.00


In [137]:
data_loader = DataLoader(dataset=dataset, batch_size=50, shuffle=True)

In [138]:
# our model, loss function, and optimizer
model = LinearNN(1, 1, 10)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [139]:
# Training loop
epochs= 2000
for epoch in range(epochs):
    for i, (inputs, labels) in enumerate(data_loader):
        
        # Forward pass: Compute predicted y by passing x to the model
        y_pred = model(inputs)
        
        # Compute and print loss
        loss = criterion(y_pred, labels)
        
        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (epoch + 1) % 100 == 0:
        print('Epoch: {}/{}, loss: {:.4f}'.format(epoch + 1., epochs, loss.data.item()))

Epoch: 100.0/2000, loss: 1.4299
Epoch: 200.0/2000, loss: 0.9665
Epoch: 300.0/2000, loss: 0.6518
Epoch: 400.0/2000, loss: 1.0255
Epoch: 500.0/2000, loss: 0.9042
Epoch: 600.0/2000, loss: 0.5256
Epoch: 700.0/2000, loss: 1.3524
Epoch: 800.0/2000, loss: 0.7821
Epoch: 900.0/2000, loss: 1.3277
Epoch: 1000.0/2000, loss: 1.1988
Epoch: 1100.0/2000, loss: 0.5334
Epoch: 1200.0/2000, loss: 1.2093
Epoch: 1300.0/2000, loss: 0.8921
Epoch: 1400.0/2000, loss: 0.7946
Epoch: 1500.0/2000, loss: 0.9578
Epoch: 1600.0/2000, loss: 1.0538
Epoch: 1700.0/2000, loss: 0.8189
Epoch: 1800.0/2000, loss: 1.2356
Epoch: 1900.0/2000, loss: 1.0036
Epoch: 2000.0/2000, loss: 1.0129


In [130]:
with torch.no_grad():
    f_temp = np.arange(212, 185, -5).reshape(-1, 1)
    y_pred = model(torch.from_numpy(f_temp).float())
    print(y_pred)

tensor([[200.8002],
        [196.0702],
        [191.3403],
        [186.6104],
        [181.8804],
        [177.1505]])


In [131]:
with torch.no_grad():
    model(torch.tensor([212]).float())